In [ ]:
import opt_note.scsp as scsp
import hexaly.optimizer
import linear_cpsat_test

In [ ]:
import marimo as mo
import nbformat

# ALPHABET から削り出すやつを Hexaly でやる

In [ ]:
class Model:
    def __init__(self, instance: list[str]):
        chars = "".join(sorted(list(set("".join(instance)))))
        max_len = len(chars) * max(len(s) for s in instance)

        hxoptimizer = hexaly.optimizer.HexalyOptimizer()
        hxmodel = hxoptimizer.model
        assert isinstance(hxoptimizer, hexaly.optimizer.HexalyOptimizer)
        assert isinstance(hxmodel, hexaly.optimizer.HxModel)

        valids = [hxmodel.bool() for _ in range(max_len)]
        cvars = [
            [
                hxmodel.int(0, max(len(_s) for _s in instance) - 1)
                for c in s
            ]
            for s in instance
        ]

        for sidx, s in enumerate(instance):
            for cidx, c in enumerate(s):
                if cidx == 0:
                    continue
                hxmodel.constraint(
                    len(chars) * cvars[sidx][cidx - 1] + chars.index(s[cidx - 1])
                    < len(chars) * cvars[sidx][cidx] + chars.index(s[cidx])
                )

        valids_array = hxmodel.array(valids)
        for sidx, s in enumerate(instance):
            for cidx, c in enumerate(s):
                hxmodel.constraint(valids_array[len(chars) * cvars[sidx][cidx] + chars.index(c)] == 1)

        hxmodel.minimize(sum(valids))
        hxmodel.close()

        self.instance = instance
        self.chars = chars
        self.hxoptimizer = hxoptimizer
        self.hxmodel = hxmodel
        self.valids = valids
        self.status: hexaly.optimizer.HxSolutionStatus | None = None

    def solve(self, time_limit: int | None = 60, log: bool = False) -> "Model":
        assert isinstance(self.hxoptimizer.param, hexaly.optimizer.HxParam)
        if time_limit is not None:
            self.hxoptimizer.param.time_limit = time_limit
        self.hxoptimizer.param.verbosity = 1 if log else 0
        self.hxoptimizer.solve()
        self.status = self.hxoptimizer.solution.status
        return self

    def to_solution(self) -> str | None:
        assert isinstance(self.hxoptimizer.solution, hexaly.optimizer.HxSolution)

        if self.status not in {
            hexaly.optimizer.HxSolutionStatus.OPTIMAL,
            hexaly.optimizer.HxSolutionStatus.FEASIBLE,
        }:
            return None

        solution = ""
        for idx, valid in enumerate(self.valids):
            if valid.value:
                solution += self.chars[idx % len(self.chars)]

        return solution

In [ ]:
def bench1(instance: list[str]) -> None:
    model = linear_cpsat_test.Model(instance).solve()
    solution = model.to_solution()
    scsp.util.show(instance)
    if solution is not None:
        scsp.util.show(instance, solution)
        print(f"solution is feasible: {scsp.util.is_feasible(instance, solution)}")
    else:
        print("--- Solution not found ---\n")

    print(f"solution status: {model.cpsolver.status_name()}")
    print(f"bset bound: {model.cpsolver.best_objective_bound}")

In [ ]:
def bench2(instance: list[str]) -> None:
    model = Model(instance).solve()
    solution = model.to_solution()
    scsp.util.show(instance)
    if solution is not None:
        scsp.util.show(instance, solution)
        print(f"solution is feasible: {scsp.util.is_feasible(instance, solution)}")
    else:
        print("--- Solution not found ---\n")

    print(f"solution is optimal: {model.hxoptimizer.solution.status == hexaly.optimizer.HxSolutionStatus.OPTIMAL}")
    print(f"bset bound: {model.hxoptimizer.solution.get_objective_bound(0)}")

In [ ]:
instance01 = scsp.example.load("uniform_q26n004k015-025.txt")

In [ ]:
bench1(instance01)

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 62) ---
 Sol: tuklcignyckosuhjmoeiqvafozppglnbrxddxbcsvrsvnhtunqgpxzvxissbxf
str1: t-k---gn--k--uh-m---------p------x----------nht--qg-xzvxis----
str2: -----i-----o---j---iq--fo----lnb-x--x-c-v-s----u-q-p--v-issbxf
str3: -u-lci-nyc-os----o---v--ozpp-l---------------------p----------
str4: -----ig-----------e--va--z--g--br-dd-bcsvr-vn---n-g----------f

solution is feasible: True
solution status: OPTIMAL
bset bound: 62.0


In [ ]:
bench2(instance01)

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 65) ---
 Sol: itkgnekulvachmopxjinyzchotqsfgovborzdppdlnbxxcvzsuqvxprvinsnsbgxf
str1: -tkgn-ku----hm-px--n---h-tq--g-------------x---z---vx---i-s------
str2: i-------------o--ji-------q-f-o---------lnbxxcv-suq--p-vi-s-sb-xf
str3: -------ul--c------iny-c-o--s--ov-o-z-pp-l------------p-----------
str4: i--g-e---va----------z-------g--b-r-d--d--b--c--s--v--rv-n-n--g-f

solution is feasible: True
solution is optimal: False
bset bound: 32


In [ ]:
instance02 = scsp.example.load("uniform_q26n008k015-025.txt")

In [ ]:
bench1(instance02)

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 103) ---
 Sol: iorxgjpwxbdeiqtvyafkopzglnrbdkruxzhlmpxcinuvyzcfhjopstumoqvdpgotxdefrzbuvxcisvcdmpsbroqvnnbibgpxflpwdeh
str1: --------------t----k---g-n---k-u--h-mpx--n------h----t---q---g--x----z--vx-is--------------------------
str2: io---j------iq----f-o---ln-b----x-----xc---v--------s-u--q--p-----------v--is-----sb-----------xf------
str3: -------------------------------u---l---cin--y-c---o-s---o-v---o------z-----------p------------p--lp----
str4: i---g------e---v-a----zg---b--r----------------------------d-----d----b---c-sv------r--vnn---g--f------
str5: ------p---------y----p--l-r------z----x---u---c----p---m-qv--g-t-d-f---u---i-vcd--sb-o-----------------
str6: ------p--bde---v------------d------

In [ ]:
bench2(instance02)

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 114) ---
 Sol: ioprtxybdjkpwxginqefkovalnrzgxbdrucdlpvcdhmpqtvxzbfgijotvxzdenrsycfhtuimqvcgsuxzbqropqvdisborsvnnozbgpxlpwdefhilps
str1: ----t-----k---g-n---k------------u-------hmp---x-------------n-----ht---q--g--xz------v---------------x-------i--s
str2: io-------j-----i-q-f-o--ln----b----------------x---------x-------c-------v--su---q--p-v-is---s-----b--x-----f-----
str3: ---------------------------------u--l--c------------i--------n--yc-----------------o-----s-o--v--oz--p--p------lp-
str4: i-------------g---e---va---zg-b-r--d----d--------b---------------c----------s---------v-----r-vnn---g-------f-----
str5: --p---y----p------------l-rz-x---uc--p----m-q-v----g---t---d------f--ui--vc------------d-s

In [ ]:
instance03 = scsp.example.load("uniform_q26n016k015-025.txt")

In [ ]:
bench1(instance03)

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 146) ---
  Sol: hklrtxioujkswxgipqbdefovajlnyzcfgpbklrsxzdxachuvcdipsuefhmnqrvzeflbcgpqsvdijqrtvwxybcenorzadhkmnstwbfkmorsuvgioquvzcdgkprsxzbjopvxbhlmwdefgimpsuwy
str01: ----t-----k---g------------n-------k----------u---------hm-----------p-----------x----n-----h----t-------------q-----g----xz----vx---------i--s---
str02: ------io-j-----i-q---fo---ln------b----x--x-c--v----su-----q---------p--v-i---------------------s--------s------------------b----x-------f--------
str03: --------u---

In [ ]:
bench2(instance03)

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 166) ---
  Sol: ehiknoprtwxybdjkpqsuwafghlrsxzeuvacdinqxyzcfgoqsuvajqtceknpuwzbfghijlmopqrdnrvabglzcptxdefiouxeiknrsvzbcdhikpstvmqsubeqrwzahknopqrtvbfhimglnpqsznsuvxzbdglvwxcdefhimsy
str01: --------t------k-------g-------------n------------------k--u-----h---m-p--------------x----------n-------h----t--q-----------------------g----------xz----v-x-----i-s-
str02: --i--o--------j---------------------i-q----f-o----------------------l------n---b------x------x---------c-------v--su--q-------

CP-SAT には勝てなそうですね...

Hexaly に関しては Weighted Majority Merge をベースにしたものの方が良さげ.

## ログを見よう

In [ ]:
instance04 = scsp.example.load("uniform_q05n010k010-010.txt")
instance05 = scsp.example.load("uniform_q05n050k010-010.txt")
instance06 = scsp.example.load("nucleotide_n010k010.txt")
instance07 = scsp.example.load("nucleotide_n050k050.txt")
instance08 = scsp.example.load("protein_n010k010.txt")
instance09 = scsp.example.load("protein_n050k050.txt")

In [ ]:
_model = Model(instance01)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Model:  expressions = 2097, decisions = 709, constraints = 164, objectives = 1
Param:  time limit = 120 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 111)


[  1 sec,  251193 itr]:           70


[  2 sec,  536299 itr]:           70


[  3 sec,  860414 itr]:           68


[  4 sec,  860414 itr]:           68


...

[117 sec, 34232359 itr]:           65


[118 sec, 34750322 itr]:           65


[119 sec, 35007121 itr]:           65


[120 sec, 35007121 itr]:           65
[ optimality gap      ]:       44.62%


[120 sec, 35244706 itr]:           65
[ optimality gap      ]:       44.62%

35244706 iterations performed in 120 seconds

Feasible solution: 
  obj    =           65
  gap    =       44.62%
  bounds =           36


In [ ]:
_model = Model(instance02)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Model:  expressions = 3038, decisions = 825, constraints = 342, objectives = 1
Param:  time limit = 120 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 262)


[  1 sec,   49921 itr]:          130


[  2 sec,  225192 itr]:          125


[  3 sec,  225192 itr]:          125


[  4 sec,  564950 itr]:          118


...

[117 sec, 20037054 itr]:          110


[118 sec, 20351231 itr]:          109


[119 sec, 20351231 itr]:          109


[120 sec, 20679198 itr]:          109
[ optimality gap      ]:       63.30%


[120 sec, 20679198 itr]:          109
[ optimality gap      ]:       63.30%

20679198 iterations performed in 120 seconds

Feasible solution: 
  obj    =          109
  gap    =       63.30%
  bounds =           40


In [ ]:
_model = Model(instance03)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Model:  expressions = 4478, decisions = 973, constraints = 630, objectives = 1
Param:  time limit = 120 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 479)


[  1 sec,  104609 itr]:          191


[  2 sec,  172981 itr]:          185


[  3 sec,  172981 itr]:          185


[  4 sec,  398815 itr]:          176


...

[117 sec, 13036580 itr]:          165


[118 sec, 13143741 itr]:          165


[119 sec, 13252927 itr]:          164


[120 sec, 13252927 itr]:          164
[ optimality gap      ]:       77.44%


[120 sec, 13359014 itr]:          164
[ optimality gap      ]:       76.83%

13359014 iterations performed in 120 seconds

Feasible solution: 
  obj    =          164
  gap    =       76.83%
  bounds =           38


In [ ]:
_model = Model(instance04)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Model:  expressions = 1058, decisions = 150, constraints = 190, objectives = 1
Param:  time limit = 120 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 144)


[  1 sec,  140372 itr]:           31


[  2 sec,  571389 itr]:           30


[  3 sec,  571389 itr]:           30


[  4 sec,  947718 itr]:           30


...

[ 32 sec, 9258300 itr]:           29


[ 33 sec, 9258300 itr]:           29


[ 34 sec, 9638417 itr]:           29


[ 35 sec, 9638417 itr]:           29


[ 35 sec, 9908565 itr]:           29
[ optimality gap     ]:           0%

9908565 iterations performed in 35 seconds

Optimal solution: 
  obj    =           29
  gap    =           0%
  bounds =           29


In [ ]:
_model = Model(instance05)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Model:  expressions = 4858, decisions = 550, constraints = 950, objectives = 1
Param:  time limit = 120 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 753)


[  1 sec,       0 itr]: No feasible solution found (infeas = 753)


[  2 sec,  121852 itr]:           38
[ optimality gap     ]:       60.53%


[  3 sec,  155923 itr]:           38


[  4 sec,  203018 itr]:           37


...

[117 sec, 6458927 itr]:           35


[118 sec, 6511783 itr]:           35


[119 sec, 6607837 itr]:           35


[120 sec, 6629077 itr]:           35


[120 sec, 6629077 itr]:           35
[ optimality gap     ]:       37.14%

6629077 iterations performed in 120 seconds

Feasible solution: 
  obj    =           35
  gap    =       37.14%
  bounds =           22


In [ ]:
_model = Model(instance06)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Model:  expressions = 1037, decisions = 140, constraints = 190, objectives = 1
Param:  time limit = 120 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 159)


[  1 sec,       0 itr]: No feasible solution found (infeas = 159)


[  2 sec,  703840 itr]:           25
[ optimality gap     ]:       44.00%


[  3 sec, 1096245 itr]:           24


[  4 sec, 1519515 itr]:           24


...

[ 47 sec, 11793914 itr]:           24


[ 48 sec, 12063470 itr]:           24


[ 49 sec, 12614934 itr]:           24


[ 50 sec, 12890989 itr]:           24


[ 50 sec, 12993031 itr]:           24
[ optimality gap      ]:           0%

12993031 iterations performed in 50 seconds

Optimal solution: 
  obj    =           24
  gap    =           0%
  bounds =           24


In [ ]:
_model = Model(instance07)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Model:  expressions = 25258, decisions = 2750, constraints = 4950, objectives = 1
Param:  time limit = 120 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 4072)


[  1 sec,   32930 itr]:          200


[  2 sec,   32930 itr]:          200


[  3 sec,   80034 itr]:          199


[  4 sec,  152246 itr]:          199


...

[117 sec, 4410477 itr]:          196


[118 sec, 4419512 itr]:          196


[119 sec, 4460305 itr]:          194


[120 sec, 4495323 itr]:          194
[ optimality gap     ]:       93.30%


[120 sec, 4495323 itr]:          194
[ optimality gap     ]:       93.30%

4495323 iterations performed in 120 seconds

Feasible solution: 
  obj    =          194
  gap    =       93.30%
  bounds =           13


In [ ]:
_model = Model(instance08)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Model:  expressions = 1351, decisions = 290, constraints = 190, objectives = 1
Param:  time limit = 120 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 100)


[  1 sec,  210981 itr]:           50


[  2 sec,  210981 itr]:           50


[  3 sec,  520703 itr]:           50


[  4 sec, 1182273 itr]:           50


...

[117 sec, 29674569 itr]:           47


[118 sec, 30156892 itr]:           47


[119 sec, 30413840 itr]:           47


[120 sec, 30657339 itr]:           47
[ optimality gap      ]:       21.28%


[120 sec, 30657339 itr]:           47
[ optimality gap      ]:       21.28%

30657339 iterations performed in 120 seconds

Feasible solution: 
  obj    =           47
  gap    =       21.28%
  bounds =           37


In [ ]:
_model = Model(instance09)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Model:  expressions = 26773, decisions = 3500, constraints = 4950, objectives = 1
Param:  time limit = 120 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 3831)


[  1 sec,   18187 itr]:          690


[  2 sec,   41130 itr]:          654


[  3 sec,   64432 itr]:          641


[  4 sec,   87468 itr]:          634


...

[116 sec, 2576212 itr]:          545


[117 sec, 2585276 itr]:          545


[118 sec, 2607059 itr]:          545


[119 sec, 2630630 itr]:          544


[120 sec, 2652753 itr]:          544
[ optimality gap     ]:      100.00%
[120 sec, 2652753 itr]:          544
[ optimality gap     ]:      100.00%

2652753 iterations performed in 120 seconds

Feasible solution: 
  obj    =          544
  gap    =      100.00%
  bounds =            0


巨大なインスタンスで試してみよう

In [ ]:
instance_large1 = scsp.example.load("nucleotide_n100k100.txt")
instance_large2 = scsp.example.load("protein_n100k100.txt")

In [ ]:
_model = Model(instance_large1)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Model:  expressions = 100709, decisions = 10600, constraints = 19900, objectives = 1
Param:  time limit = 120 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 16267)


[  1 sec,     474 itr]: No feasible solution found (infeas = 7857)


[  2 sec,    6973 itr]:          421
[ optimality gap     ]:      100.00%


[  3 sec,   16319 itr]:          391


[  4 sec,   26746 itr]:          390


...

[116 sec, 1144572 itr]:          375


[117 sec, 1153817 itr]:          375


[118 sec, 1164549 itr]:          375


[119 sec, 1174758 itr]:          375


[120 sec, 1185344 itr]:          375
[120 sec, 1185344 itr]:          375
[ optimality gap     ]:      100.00%

1185344 iterations performed in 120 seconds

Feasible solution: 
  obj    =          375
  gap    =      100.00%
  bounds =            0


In [ ]:
_model = Model(instance_large2)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Model:  expressions = 103523, decisions = 12000, constraints = 19900, objectives = 1
Param:  time limit = 120 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 15229)


[  1 sec,     106 itr]: No feasible solution found (infeas = 13673)


[  2 sec,    3182 itr]: No feasible solution found (infeas = 2492)


[  3 sec,    6541 itr]: No feasible solution found (infeas = 21)


[  4 sec,   14231 itr]:         1785
[ optimality gap     ]:      100.00%


...

[116 sec,  731193 itr]:         1510


[117 sec,  736702 itr]:         1510


[118 sec,  742021 itr]:         1510


[119 sec,  748181 itr]:         1509


[120 sec,  754085 itr]:         1509
[120 sec,  754085 itr]:         1509
[ optimality gap     ]:      100.00%

754085 iterations performed in 120 seconds

Feasible solution: 
  obj    =         1509
  gap    =      100.00%
  bounds =            0


この定式化をするんだったら CP-SAT でよいでしょう.